In [ ]:
import keras
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape, GlobalAveragePooling1D
from keras.layers import Conv2D, MaxPooling2D, Conv1D, MaxPooling1D
#from keras.utils import np_utils
import pickle
from keras.layers import LSTM
from keras.layers import BatchNormalization
from keras.models import Sequential
from keras.layers import Input,Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers.core import Dropout, Activation
from keras.layers.pooling import GlobalAveragePooling2D
from keras.models import Model
in_img = Input(shape=(3, 32, 32))
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np
from sklearn import preprocessing
from scipy import stats
from keras.engine import InputLayer
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [ ]:

def read_data(file_path):

    """
    This function reads the accelerometer data from a file
    Args:
        file_path: URL pointing to the CSV file
    Returns:
        A pandas dataframe
    """

    column_names = ['user-id',
                    'timestamp',
                    'activity',
                    'HR',
                    'HRV',
                    'raw',
                    'EDA']


    df = pd.read_csv(file_path,
                     header=None,
                     names=column_names)


    df.dropna(axis=0, how='any', inplace=True)

    return df



In [ ]:
def gen_model():
    in_img = Input(shape=(3, 32, 32))
    x = Convolution2D(12, 3, 3, subsample=(2, 2), border_mode='valid', name='conv1')(in_img)
    x = Activation('relu', name='relu_conv1')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool1')(x)
    x = Convolution2D(3, 1, 1, border_mode='valid', name='conv2')(x)
    x = Activation('relu', name='relu_conv2')(x)
    x = GlobalAveragePooling2D()(x)
    o = Activation('softmax', name='loss')(x)
    model = Model(input=in_img, output=[o])
    return model



In [ ]:

def show_basic_dataframe_info(dataframe,
                              preview_rows=20):

    """
    This function shows basic information for the given dataframe
    Args:
        dataframe: A Pandas DataFrame expected to contain data
        preview_rows: An integer value of how many rows to preview
    Returns:
        Nothing
    """

    # Shape and how many rows and columns
    print("Number of columns in the dataframe: %i" % (dataframe.shape[1]))
    print("Number of rows in the dataframe: %i\n" % (dataframe.shape[0]))
    print("First 20 rows of the dataframe:\n")
    # Show first 20 rows
    print(dataframe.head(preview_rows))
    print("\nDescription of dataframe:\n")
    # Describe dataset like mean, min, max, etc.
    # print(dataframe.describe())



In [ ]:

def plot_activity(activity, data):

    fig, (ax0, ax1, ax2) = plt.subplots(nrows=3,
         figsize=(15, 10),
         sharex=True)
    plot_axis(ax0, data['timestamp'], data['HR'], 'HR')
    plot_axis(ax1, data['timestamp'], data['HRV'], 'HRV')
    plot_axis(ax2, data['timestamp'], data['EDA'], 'EDA')
    plt.subplots_adjust(hspace=0.2)
    fig.suptitle(activity)
    plt.subplots_adjust(top=0.90)
    plt.show()


In [ ]:

def plot_axis(ax, x, y, title):

    ax.plot(x, y)
    ax.set_title(title)
    ax.xaxis.set_visible(False)
    ax.set_ylim([min(y) - np.std(y), max(y) + np.std(y)])
    ax.set_xlim([min(x), max(x)])
    ax.grid(True)



In [ ]:

def feature_normalize(dataset):

    mu = np.mean(dataset, axis=0)
    sigma = np.std(dataset, axis=0)
    return (dataset - mu)/sigma


In [ ]:


def create_segments_and_labels(df, time_steps, step, label_name):

    """
    This function receives a dataframe and returns the reshaped segments
    of x,y,z acceleration as well as the corresponding labels
    Args:
        df: Dataframe in the expected format
        time_steps: Integer value of the length of a segment that is created
    Returns:
        reshaped_segments
        labels:
    """

    # x, y, z acceleration as features
    N_FEATURES = 4
    # Number of steps to advance in each iteration (for me, it should always
    # be equal to the time_steps in order to have no overlap between segments)
    # step = time_steps
    segments = []
    labels = []
    for i in range(0, len(df) - time_steps, step):
        zs7 = df['EDA'].values[i: i + time_steps]
        zs8 = df['HR'].values[i: i + time_steps]
        zs9 = df['HRV'].values[i: i + time_steps]
        zs10 = df['raw'].values[i: i + time_steps]

        # Retrieve the most often used label in this segment
        label = stats.mode(df[label_name][i: i + time_steps])[0][0]
        segments.append([zs7, zs8, zs9, zs10])
        labels.append(label)

    # Bring the segments into a better shape
    reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, time_steps, N_FEATURES)
    labels = np.asarray(labels)

    return reshaped_segments, labels





In [ ]:


# ------- THE PROGRAM TO LOAD DATA AND TRAIN THE MODEL -------

# Set some standard parameters upfront
pd.options.display.float_format = '{:.1f}'.format
sns.set() # Default seaborn look and feel
plt.style.use('ggplot')
print('keras version ', keras.__version__)

LABELS = ["0",
          "1"]
# The number of steps within one time segment
TIME_PERIODS = 100 #was 80
# The steps to take from one segment to the next; if this value is equal to
#TIME_PERIODS, then there is no overlap between the segments
STEP_DISTANCE = 20 #was 40

# %%

print("\n--- Load, inspect and transform data ---\n")

# Load data set containing all the data from csv
#df = read_data('Data/WISDM_ar_v1.1_raw.txt')
#df = read_data('combined_clean3.csv')
df = read_data('user6.csv')
df1 = read_data('user6.csv')


# Describe the data
show_basic_dataframe_info(df, 20)

#df['activity'].value_counts().plot(kind='bar',
#                                   title='Training Examples by Activity Type')
#plt.show()

#df['user-id'].value_counts().plot(kind='bar',
#                                  title='Training Examples by User')
#plt.show()

#for activity in np.unique(df["activity"]):
#    subset = df[df["activity"] == activity][:180]
#    plot_activity(activity, subset)

# Define column name of the label vector
LABEL = "activity"
# Transform the labels from String to Integer via LabelEncoder
le = preprocessing.LabelEncoder()
# Add a new column to the existing DataFrame with the encoded values
df[LABEL] = le.fit_transform(df["activity"].values.ravel())
df1[LABEL] = le.fit_transform(df1["activity"].values.ravel())

# %%

print("\n--- Reshape the data into segments ---\n")



print("\n--- Fit the model ---\n")

# The EarlyStopping callback monitors training accuracy:
# if it fails to improve for two consecutive epochs,
# training stops early
callbacks_list = [
    keras.callbacks.ModelCheckpoint(
        filepath='best_model.{epoch:02d}-{val_loss:.2f}.h5',
        monitor='val_loss', save_best_only=True),
    keras.callbacks.EarlyStopping(monitor='acc', patience=1)
]



# Hyper-parameters
BATCH_SIZE = 400
EPOCHS = 50


#for i in range(5):

# Differentiate between test set and training set
#df_test = df[df['user-id'] <= i*4]
df_train = df
df_test = df

df_train1 = df1
df_test1 = df1


# Normalize features for training data set
#df_train['touch'] = feature_normalize(df['touch'])
#df_train['aX'] = feature_normalize(df['aX'])
#df_train['aY'] = feature_normalize(df['aY'])
#df_train['aZ'] = feature_normalize(df['aZ'])
#df_train['gX'] = feature_normalize(df['gX'])
#df_train['gY'] = feature_normalize(df['gY'])
#df_train['gZ'] = feature_normalize(df['gZ'])
#df_train['aSqrt'] = feature_normalize(df['aSqrt'])
df_train['EDA'] = feature_normalize(df['EDA'])
df_train['HR'] = feature_normalize(df['HR'])
df_train['HRV'] = feature_normalize(df['HRV'])
df_train['raw'] = feature_normalize(df['raw'])


#df_train['gX'] = feature_normalize(df['gX'])
#df_train['gY'] = feature_normalize(df['gY'])
#df_train['gZ'] = feature_normalize(df['gZ'])
#df_train['aSqrt'] = feature_normalize(df['aSqrt'])




#df_train1['touch'] = feature_normalize(df1['touch'])
#df_train1['aX'] = feature_normalize(df1['aX'])
#df_train1['aY'] = feature_normalize(df1['aY'])
#df_train1['aZ'] = feature_normalize(df1['aZ'])
#df_train1['gX'] = feature_normalize(df1['gX'])
#df_train1['gY'] = feature_normalize(df1['gY'])
#df_train1['gZ'] = feature_normalize(df1['gZ'])
#df_train1['aSqrt'] = feature_normalize(df1['aSqrt'])

df_train1['EDA'] = feature_normalize(df1['EDA'])
df_train1['HR'] = feature_normalize(df1['HR'])
df_train1['HRV'] = feature_normalize(df1['HRV'])
df_train1['raw'] = feature_normalize(df1['raw'])

#df_train1['gX'] = feature_normalize(df1['gX'])
#df_train1['gY'] = feature_normalize(df1['gY'])
#df_train1['gZ'] = feature_normalize(df1['gZ'])
#df_train1['aSqrt'] = feature_normalize(df1['aSqrt'])


# Round in order to comply to NSNumber from iOS
#df_train = df_train.round({'x-axis': 6, 'y-axis': 6, 'z-axis': 6})

# Reshape the training data into segments
# so that they can be processed by the network

x_train, y_train = create_segments_and_labels(df_train,TIME_PERIODS,STEP_DISTANCE,LABEL)

x_train, x_test1, y_train, y_test1 = train_test_split(x_train, y_train, test_size=0.3, random_state=42)


Xtest, Ytest = create_segments_and_labels(df_train1,TIME_PERIODS,STEP_DISTANCE,LABEL)



In [ ]:
df

In [ ]:
model = keras.models.load_model('user6.h5')
#model = keras.models.load_model('experiment.h5')

In [ ]:
model.summary()

In [ ]:
#model._layers.pop(0)
#model.summary()

In [ ]:
for i in range(8): #was(8)
    model.layers[i].trainable = False

In [ ]:
for i in range(4,8):
    model.layers[i].trainable = True

In [ ]:
for l in model.layers:
    print(l.name, l.trainable)

In [ ]:
#ll = model.layers[6].output #was 6
#ll = BatchNormalization(axis = -1)(ll)
#ll = Dense(32)(ll)
#ll = Dense(160)(ll) #was 64 now 160
#ll = Dropout(0.5)(ll)
#ll = Dense(2,activation="sigmoid")(ll)

ll = model.layers[6].output #was 6
ll = BatchNormalization(axis = -1)(ll)
ll = Dense(64)(ll)
ll = Dense(32)(ll) #was 64 now 160
ll = Dropout(0.5)(ll)
ll = Dense(2,activation="sigmoid")(ll)


# The number of steps within one time segment
#TIME_PERIODS = 100 #was 80
# The steps to take from one segment to the next; if this value is equal to
#TIME_PERIODS, then there is no overlap between the segments
#STEP_DISTANCE = 20 #was 40
# Set input & output dimensions
#num_time_periods, num_sensors = x_train.shape[1], x_train.shape[2]
#num_classes = le.classes_.size
print(list(le.classes_))

# Set input_shape / reshape for Keras
# Remark: acceleration data is concatenated in one array in order to feed
# it properly into coreml later, the preferred matrix of shape [40,3]
# cannot be read in with the current version of coreml (see also reshape
# layer as the first layer in the keras model)

#input_shape = (num_time_periods*num_sensors)

#inlayer = model.layers[0].input
#inlayer = Reshape((TIME_PERIODS, num_sensors), input_shape=(input_shape,)) (inlayer)


#input_layer = InputLayer(input_shape=(41182, 400), name="input_1")
#model.layers[0] = input_layer



In [ ]:
new_model = Model(inputs=model.input,outputs=ll)


In [ ]:
new_model.summary()

In [ ]:
for l in new_model.layers:
    print(l.name, l.trainable)

In [ ]:
#new_model.layers[11].get_weights()[0]

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.33)


# Accuracy on training data: 99%
# Accuracy on test data: 91%

# %%



print("\n--- Reshape data to be accepted by Keras ---\n")

# Inspect x data
print('x_train shape: ', x_train.shape)
# Displays (20869, 40, 3)
print(x_train.shape[0], 'training samples')
# Displays 20869 train samples

# Inspect y data
print('y_train shape: ', y_train.shape)
# Displays (20869,)

# Set input & output dimensions
num_time_periods, num_sensors = x_train.shape[1], x_train.shape[2]
num_classes = le.classes_.size
print(list(le.classes_))

# Set input_shape / reshape for Keras
# Remark: acceleration data is concatenated in one array in order to feed
# it properly into coreml later, the preferred matrix of shape [40,3]
# cannot be read in with the current version of coreml (see also reshape
# layer as the first layer in the keras model)
input_shape = (num_time_periods*num_sensors)
x_train = x_train.reshape(x_train.shape[0], input_shape)
x_test1 = x_test1.reshape(x_test1.shape[0], input_shape)
Xtest = Xtest.reshape(Xtest.shape[0], input_shape)
print('x_train shape:', x_train.shape)
# x_train shape: (20869, 120)
print('input_shape:', input_shape)
# input_shape: (120)

# Convert type for Keras otherwise Keras cannot process the data
x_train = x_train.astype("float32")
Xtest = Xtest.astype("float32")
y_train = y_train.astype("float32")
Ytest = Ytest.astype("float32")

x_test1 = x_test1.astype("float32")
y_test1 = y_test1.astype("float32")

y_train = np_utils.to_categorical(y_train, num_classes)
Ytest =  np_utils.to_categorical(Ytest, num_classes)
y_test1 =  np_utils.to_categorical(y_test1, num_classes)


In [ ]:
num_classes

In [ ]:
y_train

In [ ]:


def show_confusion_matrix(validations, predictions):

    matrix = metrics.confusion_matrix(validations, predictions)
    plt.figure(figsize=(6, 4))
    sns.heatmap(matrix,
                cmap="coolwarm",
                linecolor='white',
                linewidths=1,
                xticklabels=LABELS,
                yticklabels=LABELS,
                annot=True,
                fmt="d")
    plt.title("Confusion Matrix")
    plt.ylabel("True Label")
    plt.xlabel("Predicted Label")
    plt.show()



In [ ]:
#need to do model.fit

BATCH_SIZE = 400
EPOCHS = 50

new_model.compile(loss='binary_crossentropy',
                optimizer='adam', metrics=['accuracy'])


from sklearn.model_selection import KFold
kf = KFold(n_splits=10,  shuffle=True) # Define the split - into 2 folds
kf.get_n_splits(x_train) # returns the number of splitting iterations in the cross-validator
print(kf)
i=0

for train_index, test_index in kf.split(x_train):
    print("Train Index: ", train_index, "\n")
    print("Test Index: ", test_index)

    X_train, X_test, Y_train, Y_test = x_train[train_index], x_train[test_index], y_train[train_index], y_train[test_index]
#X_train, X_test, Y_train, Y_test = x_train, Xtest, y_train, Ytest

    print("Train Index: ", Y_train)

    history = new_model.fit(X_train,
                      Y_train,
                      batch_size=BATCH_SIZE,
                      epochs=EPOCHS,
                      callbacks=callbacks_list,
                      validation_split=0.2,
                      verbose=1)


    score = new_model.evaluate(X_test, Y_test, verbose=1)

    print("\nAccuracy on test data: %0.2f" % score[1])
    print("\nLoss on test data: %0.2f" % score[0])

    # %%

    print("\n--- Confusion matrix for test data ---\n")

    y_pred_test = new_model.predict(X_test)



    if i==0:
        y_pred_test_all = y_pred_test
        Y_test_all = Y_test
        X_test_all = X_test
    if i>0:
        y_pred_test_all = np.concatenate((y_pred_test_all, y_pred_test))
        Y_test_all = np.concatenate((Y_test_all, Y_test))
        X_test_all = np.concatenate((X_test_all, X_test))


    # Take the class with the highest probability from the test predictions
    max_y_pred_test = np.argmax(y_pred_test, axis=1)
    max_y_test = np.argmax(Y_test, axis=1)

    show_confusion_matrix(max_y_test, max_y_pred_test)

    # %%

    print("\n--- Classification report for test data ---\n")

    print(classification_report(max_y_test, max_y_pred_test))
    i = i +1

In [ ]:
score = new_model.evaluate(X_test_all, Y_test_all, verbose=1)

max_y_pred_test_all = np.argmax(y_pred_test_all, axis=1)
max_y_test_all = np.argmax(Y_test_all, axis=1)

show_confusion_matrix(max_y_test_all, max_y_pred_test_all)
print("\n--- Classification report for test data ---\n")

print(classification_report(max_y_test_all, max_y_pred_test_all))
print("\nAccuracy on test data: %0.3f" % score[1])
print("\nLoss on test data: %0.3f" % score[0])

In [ ]:
score = new_model.evaluate(x_test1, y_test1, verbose=1)

y_pred_test = new_model.predict(x_test1)
max_y_pred_test = np.argmax(y_pred_test, axis=1)
max_y_test = np.argmax(y_test1, axis=1)

show_confusion_matrix(max_y_test, max_y_pred_test)

    # %%

print("\n--- Classification report for test data ---\n")

print(classification_report(max_y_test, max_y_pred_test))


print("\nAccuracy on test data: %0.3f" % score[1])
print("\nLoss on test data: %0.3f" % score[0])